In [1]:
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install transformers peft datasets bitsandbytes accelerate ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [3]:
# 모델을 직접 로드하여 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-nano-2.1b-instruct")
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-nano-2.1b-instruct")

# 텍스트를 토크나이즈하여 모델 입력 형태로 변환
input_text = "Who are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# 모델을 사용하여 출력 생성
outputs = model.generate(inputs["input_ids"], max_length=100, do_sample=True)

# 출력 디코딩
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)  # 생성된 텍스트 출력


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Who are you? I can see you in my dreams.

I'm a curious dreamer, exploring the realms of the subconscious. I may not have a physical form, but I'm here to navigate your dreams and answer your questions. What's on your mind that you'd like to share with me? Are there specific queries you have in mind, or is there something you'd like to explore about dreams themselves?

I'm fascinated by the mysterious world of dreams. What's the nature of dreams


In [4]:
# 출력 개수 제한 + linear 계층만 필터링
for name, module in model.named_modules():
    if "linear" in str(type(module)).lower():
        print(name)


model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp.gate_proj
model.layers.2.mlp.up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.mlp.gate_proj
model.layers.3.mlp.up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.o_proj
model.layers.4.mlp.g

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

# 1. 모델 지정
model_id = "kakaocorp/kanana-nano-2.1b-instruct"  

# 2. 토크나이저 & 모델 불러오기 (8bit 로딩 + GPU 자동 할당)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto"
)

# 3. LoRA 학습을 위한 모델 준비
model = prepare_model_for_kbit_training(model)

# 4. Kanana-nano-2.1b-instruct 전용 target_modules로 LoRA 설정

llora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=16,  # LoRA 계수
    target_modules=[
        "self_attn.q_proj", 
        "self_attn.k_proj", 
        "self_attn.v_proj", 
        "self_attn.o_proj", 
        "mlp.gate_proj", 
        "mlp.up_proj", 
        "mlp.down_proj"
    ],  # LoRA 적용할 계층들
    lora_dropout=0.05,  # 드롭아웃 비율
    bias="none",  # 편향을 어떻게 처리할지
    task_type=TaskType.CAUSAL_LM  # 언어 모델링 작업
)

# LoRA 적용된 모델 생성
model = get_peft_model(model, llora_config)

# 학습 가능한 파라미터 출력
model.print_trainable_parameters()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 11,501,568 || all params: 2,098,480,896 || trainable%: 0.5481


In [ ]:
from datasets import load_dataset

# 1. CSV 로딩
dataset = load_dataset("csv", data_files="/app/workspace/5000_Kanana/midjourney2.csv", split="train")

# 2. 전처리 함수 정의
def format_prompt(example):
    prompt = f"### 질문: {example['long_prompt']}\n### 답변:"
    return tokenizer(prompt, text_target=example['short_prompt'], truncation=True, padding="max_length", max_length=512)

# 3. 전처리 적용
tokenized_dataset = dataset.map(format_prompt)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5058 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 1. 학습 설정
training_args = TrainingArguments(
    output_dir="/app/workspace/5000_Kanana/outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=1,
    bf16=True,
    save_strategy="no",
    report_to="none"
)

# 2. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# 3. 학습 시작
trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_502/3881944851.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.8/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to

Step,Training Loss
1,2.385300
2,3.027000
3,2.048400
4,2.712100
5,3.239600
6,2.262500
7,2.916300
8,3.179900
9,1.976700
10,2.227100


TrainOutput(global_step=3792, training_loss=2.063408225114587, metrics={'train_runtime': 14903.8239, 'train_samples_per_second': 1.018, 'train_steps_per_second': 0.254, 'total_flos': 8.707161511585382e+16, 'train_loss': 2.063408225114587, 'epoch': 2.998813760379597})

In [ ]:
model.save_pretrained("/app/workspace/5000_Kanana/outputs")
tokenizer.save_pretrained("/app/workspace/5000_Kanana/outputs")


('/app/workspace/Kanana_Prompt5000/outputs/tokenizer_config.json',
 '/app/workspace/Kanana_Prompt5000/outputs/special_tokens_map.json',
 '/app/workspace/Kanana_Prompt5000/outputs/tokenizer.json')

In [2]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. 8bit 로딩 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=True  # 핵심!
)

# 2. Base 모델 로딩
base_model = AutoModelForCausalLM.from_pretrained(
    "kakaocorp/kanana-nano-2.1b-instruct",
    device_map="auto",
    quantization_config=bnb_config
)

# 3. LoRA adapter 붙이기
model = PeftModel.from_pretrained(base_model, "/app/workspace/5000_Kanana/outputs")
model.eval()

# 4. Tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("/app/workspace/5000_Kanana/outputs", use_fast=False)

#5. 추론함수
def rewrite_prompt(input_ko):
    prompt = f"### 질문: {input_ko}\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')
    
    # num_return_sequences=1 로 설정하여 하나의 답변만 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=False,  # 샘플링 비활성화
        # temperature=0.7,  # 창의성 조절 (0.7은 예측 가능한 창의성)
        # top_p=0.9,  # 누클리어스 샘플링 (다양성 조절)
        pad_token_id=tokenizer.eos_token_id,  # 패딩을 eos_token_id로 설정
        num_return_sequences=1  # 하나의 답변만 반환
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# #5. 추론함수
# def rewrite_prompt(input_ko):
#     prompt = f"### 질문: {input_ko}\n### 답변:"
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     if 'token_type_ids' in inputs:
#         inputs.pop('token_type_ids')
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=150,
#         do_sample=True,
#         temperature=0.7,
#         top_p=0.9
#     )
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)
# # 5. 추론 함수
# def rewrite_prompt(input_ko):
#     prompt = f"### 질문:\n{input_ko}\n### 답변:"
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

#     # 불필요한 'token_type_ids' 제거
#     if 'token_type_ids' in inputs:
#         inputs.pop('token_type_ids')

#     # 출력 생성
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=150,  # 생성될 최대 토큰 수
#         do_sample=True,  # 샘플링 활성화
#         temperature=0.5,  # 창의적 출력을 유도
#         top_p=0.9,  # nucleus sampling
#         top_k=50,  # 샘플링 범위 제한
#         repetition_penalty=1.5,  # 반복 방지
#         eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
#         pad_token_id=tokenizer.pad_token_id,  # 패딩 토큰 설정
#         num_return_sequences=1,  # 반환할 시퀀스 개수
#         no_repeat_ngram_size=2  # 2-그램 반복 방지
#     )
    
#     input_len = inputs["input_ids"].shape[-1]
#     gen_ids = outputs[0][input_len:]  # 새로 생성된 부분만
#     text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

#     # "###" 이후 내용이 붙으면 잘라내기
#     if "###" in text:
#         text = text.split("###", 1)[0].strip()

#     return text

    # 결과 디코딩
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
#return output_text

In [ ]:
# 5. 테스트!
example = "창문 너머를 바라보는 고양이"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 창문 너머를 바라보는 고양이
### 답변: A majestic and serene scene unfolds: a fluffy feline, adorned in a luxurious coat of white fur, stands in a window's frame, gazing out at the night sky. The starry canvas, a canvas of deep blues and purples, stretches into the infinite, while the moon, a glowing orb, casts an ethereal light upon the scene. The cat, with its piercing green eyes, seems to be lost in contemplation, its fur a contrast to the celestial backdrop. The window, a portal to the cosmos, is adorned with intricate patterns, adding to the ethereal ambiance. This moment, a snapshot of tranquility, captures the cat's serene presence, a symbol of calm amidst the celestial dance above.
### 설명: This highquality image showcases a majestic and serene scene: a fluffy white cat, adorned in a luxurious fur coat, stands in a window's frame, gazing out at the night sky. The starry canvas, a canvas of deep blues and purples, stretches into the infinite, with the moon, a glowing orb, casting

In [13]:
# 5. 테스트!
example = "현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘
### 답변: "Create a detailed, modern and elegant building illustration. The building should be a fusion of contemporary design and architectural sophistication. It should have a sleek, streamlined shape, with a combination of glass, steel, and concrete materials. The exterior should feature a variety of windows and doors, adding to the openness and functionality of the space. The roof should be sleek and flat, with a modern solar panel array. The overall effect should be modern, elegant, and contemporary, with a focus on the overall design and functionality of the building."
### 답변: "Generate a hyperrealistic image of a modern, sleek building. The building should be a masterpiece of contemporary architecture, with a sleek, streamlined shape that exudes elegance. The exterior should feature a variety of windows and doors, adding to the openness and functionality of the space. The roof should be sleek and flat, with a modern solar panel arr

In [3]:
# 5. 테스트! *** maxtoken 300
example = "현대식 화려한 건물을 사진처럼 현실감있게 그리고 싶은데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 현대식 화려한 건물을 사진처럼 현실감있게 그리고 싶은데 미드저니 프롬프트 작성해줘
### 답변: "Create a hyperrealistic, modern and luxurious building in the style of a highend art museum. The building should have a sleek, geometric shape with large windows and a high ceiling. The exterior should be made of polished concrete and glass, with a polished metal roof. The interior should have large, open spaces with high ceilings and large windows that offer breathtaking views of the city. The design should be inspired by the works of famous artists, with large-scale murals and sculptures on the walls. The lighting should be soft and warm, creating a sense of luxury and sophistication. The colors should be bold and vibrant, with a focus on deep blues, purples, and golds. The overall effect should be one of grandeur and beauty, with a sense of awe at the sheer scale and beauty of the building."
### 답변: "Generate a hyperrealistic, modern and luxurious building in the style of a highend art museum. The building should 

In [15]:
# 5. 테스트!
example = "현대식 화려한 건물을 사진처럼 현실감있게 그리고 싶은데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 현대식 화려한 건물을 사진처럼 현실감있게 그리고 싶은데 미드저니 프롬프트 작성해줘
### 답변: "Create a hyperrealistic, modern and luxurious building in the style of a highend art museum. The building should have a sleek, geometric shape with large windows and a high ceiling. The exterior should be made of polished concrete and glass, with a polished metal roof. The interior should have large, open spaces with high ceilings and large windows that offer breathtaking views of the city. The design should be inspired by the works of famous artists, with large-scale murals and sculptures on the walls. The lighting should be soft and warm, creating a sense of luxury and sophistication. The colors should be bold and vibrant, with a focus on deep blues, purples, and golds. The overall effect should be one of grandeur and beauty, with a sense of awe at the sheer scale and beauty of the building."
### 답변: "Generate a hyperrealistic, modern and luxurious building in the style of a highend art museum. The building should 

In [16]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람을 그릴 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람을 그릴 미드저니 프롬프트 작성해줘
### 답변: "A person is playing the piano in a library. The scene is set in a welllit, wellorganized library with bookshelves lined along the walls. The person sits at a comfortable desk, holding a piano on their left, while the books stack on their right. The atmosphere is serene and quiet, with the soft clacking of the bookshelves and the occasional rustle of pages. The lighting is soft and warm, casting long shadows that add depth and dimension to the scene. The person is lost in their music, their fingers moving deftly across the keys, creating a beautiful melody that fills the library with a sense of calm and tranquility. The scene is a moment of peace and relaxation, a reminder of the calming and enriching effects of literature and music."
### 답변: "Create a detailed 3D render of a person playing the piano in a welllit library. The library is wellorganized with bookshelves lined along the walls. The person sits at a comfortable des

In [17]:
# 5. 테스트!
example = "풀숲에서 똥 싸다가 눈마주쳐서 놀란 고양이 그려줘줘줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 풀숲에서 똥 싸다가 눈마주쳐서 놀란 고양이 그려줘줘줘
### 답변: 고양이가 풀숲에서 똥을 싸다가 갑자기 눈을 마주쳐 깜짝 놀란 표정을 그린 그림입니다. 풀숲의 자연 속에서 고양이가 자연스럽게 움직이고 있으며, 고양이의 귀여운 표정과 몸짓이 생생하게 표현되어 있습니다. 풀숲의 나무와 잎사귀들이 배경에 그려져 있어, 고양이가 자연 속에서 생활하고 있다는 느낌을 줍니다. 고양이의 털은 부드럽고 자연스럽게 표현되어 있으며, 고양이의 털결이 살아있어, 실제로 보는 것 같은 생생한 느낌을 줍니다. 고양이의 눈은 놀란 표정으로 그려져 있으며, 눈동자가 동그랗게 그려져 있어, 고양이가 깜짝 놀란 모습을 잘 표현하고 있습니다. 고양이의 코와 입, 귀 등이 세밀하게 그려져 있어, 고양이의 귀여운 모습을 잘 표현하고 있습니다. 전체적으로 고양이가 풀숲에서 자연스럽게 움직이고 있으며, 고양이의 귀여운 표정과 몸짓이 생생하게 표현되어 있는 그림입니다. 고양이의 털결이 살아있어, 실제로 보는 것 같은 생생한 느낌을 줍니다. 고양이의 눈


In [18]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘
### 답변: "도서관에서 피아노를 치고 있는 사람을 그려주세요. 조용한 분위기 속에서 피아노 소리가 울려 퍼지는 장면을 상상해 주세요. 도서관의 고요한 환경과 피아노의 따뜻한 음색이 어우러진 장면을 그리며, 피아노를 치고 있는 사람의 모습을 세밀하게 표현해 주세요. 피아노를 치는 손의 움직임과 표정, 그리고 도서관의 책장 사이로 비치는 빛의 반짝임까지 세세하게 묘사해 주세요. 피아노 소리가 도서관 전체에 퍼져서 사람들의 집중을 끌어모으는 장면을 떠올리며, 그 순간의 감동과 아름다움을 담아주세요."
### 답변: "도서관에서 피아노를 치고 있는 사람을 그려주세요. 조용한 분위기 속에서 피아노 소리가 울려 퍼지는 장면을 상상해 주세요. 도서관의 고요한 환경과 피아노의 따뜻한 음색이 어우러진 장면을 그리며, 피아노를 치고 있는 사람의 모습을 세밀하게 표현해 주세요. 피아노를 치는 손의 움직임과 표정, 그리고 도서관의 책장 사이로 비치는 빛의 반짝임


In [19]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습
### 답변: A whimsical and vibrant illustration of five playful baby cats, leaping into the sky during a skydiving adventure. The cats, with their bright fur and curious expressions, are captured mid-air, creating a celestial scene with their own unique forms. The background is a deep blue, emphasizing the vivid colors of the cats and the stars above.
### 답변: A detailed digital illustration of a group of five baby cats skydiving together, their fur a kaleidoscope of colors. The cats, with their wide eyes and playful expressions, are depicted in mid-air, surrounded by a starry sky and a dark blue background, creating a sense of freedom and adventure.
### 답변: A captivating digital illustration of five baby cats skydiving together, their fur a vibrant mix of pinks, oranges, and blues. The cats, with their wide eyes and playful expressions, are shown in mid-air, surrounded by a starry sky and a dark blue background, creating a sense of fre

In [20]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘
### 답변: "Create a breathtaking scene of five baby kittens performing a skydiving jump together, their fur shimmering in the sunlight as they leap from a high altitude. As they fall, their playful forms transform into a celestial canvas of stars and clouds, with each kitten's unique expression and body language capturing the thrill of the moment. The setting sun casts a warm glow on the scene, highlighting the kittens' vibrant colors and adding a sense of adventure and freedom to the image."
### 답변: "Generate a captivating image of five baby kittens performing a skydiving jump together, their fur shining in the sunlight as they leap from a high altitude. As they fall, their playful forms transform into a celestial canvas of stars and clouds, with each kitten's unique expression and body language capturing the excitement of the moment. The setting sun casts a warm glow on the scene, highlighting the kittens' vi

In [21]:
# 5. 테스트!
example = "지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘
### 답변: Create a detailed illustration of a Jeep vehicle, featuring a majestic giraffe with vibrant, colorful fur and piercing blue eyes. The giraffe is adorned in a stylish sunglasses, capturing a unique and playful scene. The illustration should showcase the giraffe standing on the road, driving the Jeep, with the vehicle's sleek lines and vibrant colors contrasting the giraffes majestic presence. The atmosphere should be whimsical and fun, capturing the essence of a playful and adventurous moment.
### 답변: Create a vibrant and energetic illustration of a giraffe driving a Jeep in a lively scene. The giraffe, with its bright fur and piercing blue eyes, is wearing a stylish sunglasses, adding to its playful demeanor. The Jeep, with its bold colors and modern design, is parked on a sunny road, surrounded by lush greenery and a bright sky. The illustration should capture the giraffes charming and adventurous spirit, a

In [22]:
# 5. 테스트!
example = "검정색 고양이가 기지개를 펴고 있는 모습을 만화 형식으로 그려줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 검정색 고양이가 기지개를 펴고 있는 모습을 만화 형식으로 그려줘
### 답변: 검정색 고양이가 기지개를 펴고 있는 모습을 만화 형식으로 그린 그림입니다. 고양이는 귀여운 얼굴에 검정색 털을 가지고 있으며, 팔과 다리를 쭉 펴고 있는 모습입니다. 기지개를 펴는 모습을 표현하기 위해 팔과 다리를 최대한 길게 그렸으며, 고양이의 귀여운 표정과 몸짓을 통해 생동감을 더했습니다. 만화 형식으로 그려서 캐릭터의 특징을 강조하고, 독자들이 쉽게 이해할 수 있도록 표현했습니다. 4K 고해상도로 그려서 디테일이 살아있고, 색감도 선명하게 표현했습니다.
### 답변: 검정색 고양이가 기지개를 펴고 있는 모습을 만화 형식으로 그린 그림입니다. 고양이는 귀여운 얼굴에 검정색 털을 가지고 있으며, 팔과 다리를 쭉 펴고 있는 모습입니다. 기지개를 펴는 모습을 표현하기 위해 팔과 다리를 최대한 길게 그렸으며, 고양이의 귀여운 표정과 몸짓을 통해 생동감을 더했습니다. 만화 형식으로 그려서 캐릭터의 특징을 강조하고, 독자들이 쉽게 이해할 수 있도록 표현했습니다. 4K 고해상도로 그


In [24]:
# 5. 테스트!
example = "하얀색 사자가 화장실에서 샤워하면서 노래부르는 모습을 현실감 있게 그리게 프롬프트 작성"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 하얀색 사자가 화장실에서 샤워하면서 노래부르는 모습을 현실감 있게 그리게 프롬프트 작성
### 답변: A detailed digital illustration of a white lion in a bathroom, showering with a song playing in the background. The focus is on the lion's expressive pose and the realistic details of the bathroom setting, capturing a unique moment of tranquility.
### 답변: A detailed digital illustration of a white lion in a bathroom, showering with a song playing in the background. The focus is on the lion's expressive pose and the realistic details of the bathroom setting, capturing a unique moment of tranquility.
### 답변: A detailed digital illustration of a white lion in a bathroom, showering with a song playing in the background. The focus is on the lion's expressive pose and the realistic details of the bathroom setting, capturing a unique moment of tranquility.
### 답변: A detailed digital illustration of a white lion in a bathroom, showering with a song playing in the background. The focus is on the lion's expressive pose and t

In [26]:
# 5. 테스트!
example = "펭귄들이 할로윈데이를 맞이해 잭오랜턴을 뒤집어 쓴 모습을 만화 형식으로 재밌고 밝은 느낌으로 그려줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 펭귄들이 할로윈데이를 맞이해 잭오랜턴을 뒤집어 쓴 모습을 만화 형식으로 재밌고 밝은 느낌으로 그려줘
### 답변: 펭귄들이 할로윈데이를 맞아 잭오랜턴을 뒤집어쓰고 웃고 있는 모습을 만화 형식으로 그려보았어요. 각 펭귄의 개성 있는 표정과 몸짓이 살아있어서 정말 재밌어요. 배경은 할로윈 분위기를 살리기 위해 검은색과 주황색을 섞어 그렸고, 잭오랜턴은 주황색으로 표현했어요. 펭귄들의 발그레한 볼과 귀여운 다리도 잘 그려냈답니다. 만화 특유의 간결하고 재밌는 표현이 돋보이네요! 할로윈데이의 즐거운 분위기가 잘 전달되었으면 좋겠어요.
### 답변: 펭귄들이 할로윈데이를 맞아 잭오랜턴을 뒤집어쓰고 웃고 있는 모습을 만화 형식으로 그려보았어요. 각 펭귄의 개성 있는 표정과 몸짓이 살아있어서 정말 재밌어요. 배경은 할로윈 분위기를 살리기 위해 검은색과 주황색을 섞어 그렸고, 잭오랜턴은 주황색으로 표현했어요. 펭귄들의 발그레한 볼과 귀여운


In [ ]:
# 5. 테스트!
example = "펭귄들이 할로윈데이를 맞이해 잭오랜턴을 뒤집어 쓴 모습을 만화 형식으로 재밌고 밝은 느낌으로 그려줘"
print("💬 결과:\n", rewrite_prompt(example))